### Tensorflow - Notebook -2



***

The CovNet - The Convolutional Neural Network (CNN)
Main Source - https://www.tensorflow.org/tutorials/deep_cnn


***
#### Index :

1. Whats the CNN?
2. Why Python using Tensorflow? What other options for CNN.
3. WIP ...

***
#### Other Sources :

1. Yoon Kim - Convolutional Neural Networks for Sentence Classification - https://arxiv.org/pdf/1408.5882.pdf
2. 


In [1]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

/home/dhankar/anaconda2/envs/tensor/bin/python
3.6.2 |Continuum Analytics, Inc.| (default, Jul 20 2017, 13:51:32) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
sys.version_info(major=3, minor=6, micro=2, releaselevel='final', serial=0)


In [10]:
import tensorflow as tf
import numpy as np 


In [11]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

# if Batch Size == 5 , which means FIVE Images 
# Each Image has 28X28 Pixels == 784 Pixels 
# Thus TOTAL = 784 X 5 == 3920 Pixels


  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, 
        rate=0.4,   ## 40% Droput Regularization - 40% of elements randomly dropped out during training.
        training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=10) 
  ## Units 10 - as Digits can have Values == 0 to 9 

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
    
# Within - predictions, above - the   tf.argmax == Here, we want to find the largest 
# value along the dimension with index of 1, 
# which corresponds to our predictions 
# (recall that our logits tensor has shape [batch_size, 10]). 
# Within the DATA STRUCTURE - [batch_size, 10] the 10 or INDEX =1 is the Index of Predictions 
# From within this Index of Predictions  we need the MAX Arguments == Which will be CLASSES


  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


In [18]:
def main(): #unused_argv
      # Load training and eval data
      mnist = tf.contrib.learn.datasets.load_dataset("mnist")
      train_data = mnist.train.images # Returns np.array
      train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
      eval_data = mnist.test.images # Returns np.array
      print(type(eval_data))
      eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
    # Create the Estimator
      mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir="/home/dhankar/Desktop/mnist_convnet_model")
      tensors_to_log = {"probabilities": "softmax_tensor"}
      logging_hook = tf.train.LoggingTensorHook(tensors = tensors_to_log, every_n_iter = 50)
        
        # TRAIN the MODEL 
        
      train_input_fn = tf.estimator.inputs.numpy_input_fn(x = {"x": train_data}, y = train_labels, batch_size = 100, num_epochs = None, shuffle = True)
      mnist_classifier.train(input_fn = train_input_fn, steps = 20000, hooks = [logging_hook])
        
        # Evaluate the model and print results
        
      eval_input_fn = tf.estimator.inputs.numpy_input_fn(x = {"x": eval_data}, y = eval_labels, num_epochs =1, shuffle = False)
      eval_results = mnist_classifier.evaluate(input_fn = eval_input_fn)
      print(eval_results)


In [19]:
main()

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
<class 'numpy.ndarray'>
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/home/dhankar/Desktop/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7ac3ef3400>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /home/dhankar/Desktop/mnist_convnet_model/model.ckpt.
INFO:tensorflow:probabilities = [[

KeyboardInterrupt: 